## Import data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Look at data

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

Choose to use PassengerID, Survived, Pclass, Sex, Age, SibSp, Parch for the algortithm. 

For all chosen features execept for Sex, the data type is a real number. Possible problem with including a string in the classification algorithm?


In [3]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

## Look at % of passenger class that survived

In [4]:
fclass = train_data.loc[train_data.Pclass == 1]["Survived"]
rate_fclass = sum(fclass)/len(fclass)
print("% of first class who survived:", rate_fclass)

sclass = train_data.loc[train_data.Pclass == 2]["Survived"]
rate_sclass = sum(sclass)/len(sclass)
print("% of second class who survived:", rate_sclass)

tclass = train_data.loc[train_data.Pclass == 3]["Survived"]
rate_tclass = sum(tclass)/len(tclass)
print("% of third class who survived:", rate_tclass)

In [5]:
# check for null values by checking if equal to itself 
train_data.Age[888] == train_data.Age[888]

Was going to check null values like this, better to use .info

## Determine the amount of null values

In [10]:
train_data.info(show_counts=True)

## Get what % of all people that survived

In [11]:
survived = train_data.loc[train_data.Survived == 1]
nsurvived = train_data.loc[train_data.Survived == 0]
print("% of people who survived:", len(survived) / (len(survived) + len(nsurvived)))

## Check Sex data

In [12]:
train_data

## Map all 'male' to 1 and all 'female' to zero for use in logistic regression

To use logistic regression on the data set, I will need to convert all Sex variables to numeric values. Thus, I will map all 'male' to 1 and all 'female' to 0.

In [13]:
for i in range(len(train_data.Age)):
    
    if train_data.Sex[i] == 'male':
        train_data.Sex[i] = 1
    
    elif train_data.Sex[i] == 'female':
        train_data.Sex[i] = 0

## Check that data has been changed

In [14]:
train_data

In [15]:
train_data.shape

## I now excise NaN ages

In [16]:
for i in range(len(train_data.Age)):
    
    if train_data.Age[i] != train_data.Age[i]: # if we get a nan value
        train_data = train_data.drop([i], axis = 0)  # delete the row

In [17]:
train_data

## I now save the feature data as a txt file for use in an ML Octave algorithm

In [ ]:
# i now save my csv data to a text file as all data processing has been done
np.savetxt('titanic_train_data.txt',train_data.iloc[:, [2,4,5,6,7,1]].values)

# i realize above that I the survivor ID should not be included in the data to be passed
# to the algorithm, and that past algorithms that I have used put the prediction variable at the end of the
# data.

In [18]:
train_data.iloc[:, [2,4,5,6,7,1]]

## I now move to writing a logistic regression algorthim in Octave

# Looking at test data

In [20]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data

## Check nan values for all columns to be used

In [21]:
test_data.info(show_counts=True)

In [22]:
print(str(86/418 * 100) + '% of all age data is vacant')

To deal with the fact that 20% of all age data is vacant, and the fact that I do not want to guess for 20% of the data, I will instead replace all 20% with new values. Likely drawn randomly from a distribution follow that of the test data distribution of ages.

## Looking at the train data distrubtiuon of ages to input into test data so that they may be used in the algorithm

In [23]:
# i check a histogram to get the idea of age distribution
values = plt.hist(train_data.Age, bins = 20)

## Here I try a normal distribution

In [24]:
# I want to draw values from a Normal distribution to try to match the histogram
test_values = np.random.normal(20,10,714)

In [25]:
plt.hist(train_data.Age, bins = 20)
plt.hist(np.abs(test_values),bins = 20)

## Here I try random.choice, and input a function following the original distribution

In [26]:
plt.plot(values[1][:20], values[0])

In [27]:
# here i fit a cubic function to the data because it kinda resembles one
coefs = np.polyfit(values[1][:20], values[0], deg = 5) 

In [28]:
# here i generate lots of points that i will be able to choose randomly from 
x = np.linspace(0,75,1000)

# now i feed these points into my function
y = (coefs[0] * x**5) + (coefs[1] * x**4) + (coefs[2] * x**3) + (coefs[3] * x**2) + (coefs[4] * x) + coefs[5]

In [29]:
# here i plot the distribution that I will pull random values from to the actual training distriubtion of ages
plt.plot(x,y)
plt.plot(values[1][:20], values[0])

## I conclude that a fifth order polynomail fits the data well enough to draw random ages from to fill the 86 null values in the testing age

## Overnight I realized that the better thing to do would be to pull from the distribution of Ages given in the training set

# Recap

### I am tired so I recap. I have done the following:

1. Chosen which features to use based on historical societal mindsets, pre-modelling analysis, and intuition. These features are: PassengerClass, Age, Sex, Parents, and Siblings (and of course survival for training data).
2. Determined the data type of all features and the amount of null values in each feature.
3. Changed the sex feature from string data types to numerical data type so that they may be used in an algorithm.
4. Excised all rows that contined NaN values from the training set - a total of 177 / 891 of Age data or 20% of the training data. (This may be changed later)
5. Saved the features that I plan on using from a .csv file to a .txt file to run with Octave 
6. Analyed the testing data, and found that 20% of testing Ages were null. 20% is too high to throw out the window, so I resolved to finding a way to input random ages to the null locations.
7. Determined that a fifth order polynomial fits the training Age distribution well, and constructed a distribution of values following such a polynomial to draw from to input into null locations.
8. Modified an Octave regularized polynomial logistic regression algorithm to find the optimal parameters for a model that will predict the survivability of a new Titanic passenger based on the features that I have chosen.
9. Filled in null ages values in the test data set from the fifth order function that I have created with random.choice(distribution). The function needs to be made into either a probability distribution, or into an actul distribution. Currently it is just a linear set of values. INSTEAD PULL FROM AGES
10. Finsh modifiying the classification algorithm: encode the ability to pass an array of data and get back the predicted survival array that can be submitted to Kaggle

### When I am not tired I need to do:

3. Run the program
4. Fix bugs.
5. Check some stuff, like if women, childern, and first class passengers are predicted to survive more.
6. Submit.

## Here I fill in values in the testing set with randomly pulled Ages from the training set (I believe that it is better not to do this with the training set as there are lots of values left after excision ~700, and excising ensure that all true values are used)

In [ ]:
# loop through testing set ages and replace when we get a nan
for i in range(len(test_data.Age)):
    
    if test_data.Age[i] != test_data.Age[i]: # if we get a nan value
        test_data.Age[i] = np.random.choice(train_data.Age)  # replace with randomly chosen value from training data

In [ ]:
# looks good
test_data

## I change the age values from strings to numerals as before

In [ ]:
for i in range(len(test_data.Age)):
    
    if test_data.Sex[i] == 'male':
        test_data.Sex[i] = 1
    
    elif test_data.Sex[i] == 'female':
        test_data.Sex[i] = 0

In [ ]:
# check that values have been changed
test_data

## I save the testing data to a txt for use with Octave

In [ ]:
# survivor ID should is no included in the data to be passed to the algorithm, 
# and past algorithms that I have used put the prediction variable at the end of the data.
np.savetxt('titanic_test_data.txt',test_data.iloc[:, [1,3,4,5,6]].values)

In [ ]:
test_data.iloc[:,[1,3,4,5,6]]

In [ ]:
test_data.PassengerId

# Below I make a data set that only contains two features to use for Octave processing

### I will try using two features, Pclass and Gender, on the first iteration


In [ ]:
train_data.iloc[:, [2,4,1]]

In [ ]:
# survivor ID should is no included in the data to be passed to the algorithm, 
# and past algorithms that I have used put the prediction variable at the end of the data.
np.savetxt('tit_train_pclass_sex.txt',train_data.iloc[:, [2,4,1]].values)

In [ ]:
# now I check the testing data
test_data.iloc[:, [1,3]]

In [ ]:
np.savetxt('tit_test_pclass_sex.txt',test_data.iloc[:, [1,3]].values)

# Three Feature Set Generation Area

### Pclass, Gender, Parents

In [ ]:
train_data.iloc[:, [2,4,7,1]]

In [ ]:
np.savetxt('train_class_sex_par.txt',train_data.iloc[:, [2,4,7,1]].values)

In [ ]:
test_data.iloc[:, [1,3,6]]

In [ ]:
np.savetxt('test_class_sex_par.txt',test_data.iloc[:, [1,3,6]].values)

### Want to get a sense of whether parents or siblings would implicate survial more

In [30]:
par = train_data.loc[train_data.Parch != 0]["Survived"]
rate_par = sum(par)/len(par)
print("% of people with parents on board who survived:", rate_par)

nopar = train_data.loc[train_data.Parch == 0]["Survived"]
rate_nopar = sum(nopar)/len(nopar)
print("% of people without parents on board who survived:", rate_nopar)

sib = train_data.loc[train_data.SibSp != 0]["Survived"]
rate_sib = sum(sib)/len(sib)
print("% of people with siblings on board who survived:", rate_sib)

nosib = train_data.loc[train_data.SibSp == 0]["Survived"]
rate_nosib = sum(nosib)/len(nosib)
print("% of people without siblings on board who survived:", rate_nosib)